In [2]:
import pandas as pd
import streamlit as st
import sqlite3
import pydeck as pdk

In [22]:
# Connexion à la base de données
connexion = sqlite3.connect('bdd.db')

# Récupération des données de la table objet
query_objet = "SELECT * FROM objet"
df_objet = pd.read_sql_query(query_objet, connexion)

# Récupération des données de la table gare
query_gare = "SELECT * FROM gare"
df_gare = pd.read_sql_query(query_gare, connexion)

# Fermeture de la connexion à la base de données
connexion.close()

# Sélection de l'année et du type d'objet avec des widgets Streamlit
annee = '2019'

types = df_objet['type'].unique().tolist()
types_selectionnes = st.multiselect("Sélectionnez le type d'objet :", types, default=types[:], key=3)

# Filtrage des données en fonction de la sélection de l'utilisateur
df_objet_filtre = df_objet[(df_objet['type'].isin(types_selectionnes)) & (df_objet['date'].str[-4:] == '2019')]
df_gare_filtre = df_gare[['gare', 'lat', 'lon', 'frequentation_' + annee]]
df_gare = df_gare.sort_values('gare').reset_index().drop(columns='index')
# Calcul du nombre d'objets trouvés des types sélectionnés pour chaque gare
df_nb_objets_par_gare = df_objet_filtre.groupby('gare')['id'].count().reset_index().rename(columns={'id': 'nb_objets'})
df_gare



,gare,lat,lon,frequentation_2019,frequentation_2020,frequentation_2021,frequentation_2022
0,Paris Austerlitz,48.842285,2.364891,44105915,26806157,45096165,45096166
1,Paris Bercy,48.839170,2.382780,8800506,5091294,6822335,6822336
2,Paris Est,48.876742,2.358424,93300618,48823493,62800592,62800593
3,Paris Gare de Lyon,48.844888,2.373520,252766189,123698547,180857682,180857683
4,Paris Gare du Nord,48.880185,2.355151,517658380,199902302,275471366,275471367
5,Paris Montparnasse,48.841172,2.320514,137996789,73054629,92450093,92450094
6,Paris Saint-Lazare,48.876242,2.325331,250460982,267092898,213505115,213505116


In [113]:
def load_data():
    conn = sqlite3.connect("bdd.db")
    query = """
    SELECT objet.date, objet.gare, objet.type, objet.nature, gare.lat, gare.lon, temperature.temperature
    FROM objet
    JOIN gare ON objet.gare = gare.gare
    JOIN temperature ON objet.date = temperature.date
    """
    df = pd.read_sql_query(query, conn)
    conn.close()
    return df

def prepare_data(df):
    df['nombre_objets'] = 1
    df_filtre = df.groupby(['gare', 'temperature'])['nombre_objets'].sum().reset_index()
    return df_filtre

st.title("Analyse des objets trouvés en fonction de la température")

df = load_data()
df_filtre = prepare_data(df)
df_filtre

,gare,temperature,nombre_objets
0,Paris Austerlitz,-3.5,1
1,Paris Austerlitz,-2.8,2
2,Paris Austerlitz,-2.6,1
3,Paris Austerlitz,-2.3,4
4,Paris Austerlitz,-2.0,8
...,...,...,...
1862,Paris Saint-Lazare,28.8,11
1863,Paris Saint-Lazare,29.3,5
1864,Paris Saint-Lazare,30.4,8
1865,Paris Saint-Lazare,30.7,15


In [2]:
import colorcet as cc

def plasma_color(value, min_value, max_value):
    value_normalized = (value - min_value) / (max_value - min_value)
    color_index = int(value_normalized * 255)
    return list(cc.palette["CET_L8"][color_index])

plasma_color(100,100,10)

['#', '0', '0', '0', 'e', '5', 'c']

In [6]:
import sqlite3
import pandas as pd
def load_data():
    conn = sqlite3.connect("bdd.db")
    query = """
    SELECT objet.date, objet.gare, objet.type, objet.nature, gare.lat, gare.lon, temperature.temperature
    FROM objet
    JOIN gare ON objet.gare = gare.gare
    JOIN temperature ON objet.date = temperature.date
    """
    df = pd.read_sql_query(query, conn)
    conn.close()
    return df

def prepare_data(df):
    df['nombre_objets'] = 1
    df_filtre = df.groupby(['gare', 'temperature'])['nombre_objets'].sum().reset_index()
    return df_filtre


df = load_data()
df_filtre = prepare_data(df)
df

,date,gare,type,nature,lat,lon,temperature,nombre_objets
0,21/11/2019,Paris Gare de Lyon,"Bagagerie: sacs, valises, cartables","Valise, sac sur roulettes",48.844888,2.37352,5.2,1
1,21/11/2019,Paris Gare de Lyon,"Appareils électroniques, informatiques, appare...","Téléphone portable protégé (étui, coque,…)",48.844888,2.37352,5.2,1
2,21/11/2019,Paris Gare de Lyon,"Appareils électroniques, informatiques, appare...","Téléphone portable protégé (étui, coque,…)",48.844888,2.37352,5.2,1
3,21/11/2019,Paris Gare de Lyon,"Appareils électroniques, informatiques, appare...","Téléphone portable protégé (étui, coque,…)",48.844888,2.37352,5.2,1
4,21/11/2019,Paris Gare de Lyon,"Appareils électroniques, informatiques, appare...","Téléphone portable protégé (étui, coque,…)",48.844888,2.37352,5.2,1
...,...,...,...,...,...,...,...,...
81627,17/04/2022,Paris Bercy,"Bagagerie: sacs, valises, cartables","Valise, sac sur roulettes",48.839170,2.38278,11.8,1
81628,20/04/2022,Paris Bercy,"Vélos, trottinettes, accessoires 2 roues",Accessoires 2 roues,48.839170,2.38278,12.4,1
81629,26/04/2022,Paris Bercy,"Clés, porte-clés, badge magnétique",Clé de voiture,48.839170,2.38278,10.0,1
81630,29/04/2022,Paris Bercy,"Bagagerie: sacs, valises, cartables","Sac de voyage, sac de sport, sac à bandoulière",48.839170,2.38278,13.6,1


In [41]:
nbrr = df.nombre_objets[(df.gare == 'Paris Bercy') & (df.date.str[-4:] == '2019')].count()
nbrr*1_000_000/8_800_506
nbrr

570

In [47]:
# Connexion à la base de données
connexion = sqlite3.connect('bdd.db')

# Récupération des données de la table objet
query_objet = "SELECT * FROM objet"
df_objet = pd.read_sql_query(query_objet, connexion)

# Récupération des données de la table gare
query_gare = "SELECT * FROM gare"
df_gare = pd.read_sql_query(query_gare, connexion)

# Fermeture de la connexion à la base de données
connexion.close()


annee_selectionnee = '2019'

types = df_objet['type'].unique().tolist()
types_selectionnes = [types[:]]

# Filtrage des données en fonction de la sélection de l'utilisateur
df_objet_filtre = df_objet[(df_objet['type'].isin(types_selectionnes)) & (df_objet['date'].str[-4:] == annee_selectionnee)]
df_gare_filtre = df_gare[['gare', 'lat', 'lon', 'frequentation_' + annee_selectionnee]]

# Calcul du nombre d'objets trouvés des types sélectionnés pour chaque gare
df_nb_objets_par_gare = df_objet_filtre.groupby('gare')['id'].count().reset_index().rename(columns={'id': 'nb_objets'})

# Calcul du ratio fréquentation / nombre d'objets trouvés
df_gare['ratio_freq_objets'] = round((df_nb_objets_par_gare.nb_objets * 1_000_000) / df_gare['frequentation_' + annee_selectionnee])

import colorcet as cc

def plasma_color(value, min_value, max_value):
    value_normalized = (value - min_value) / (max_value - min_value)
    color_index = int(value_normalized * 255)
    hex_color = cc.palette["CET_CBD2"][color_index]
    rgb_color = tuple(int(hex_color[i:i+2], 16) for i in (1, 3, 5))
    return list(rgb_color)

# Calcul du ratio fréquentation / nombre d'objets trouvés
df_gare['ratio_freq_objets'] = round((df_nb_objets_par_gare.nb_objets * 1_000_000 / df_gare['frequentation_' + annee_selectionnee]))
df_nb_objets_par_gare

,gare,nb_objets


In [36]:
df_gare

,gare,lat,lon,frequentation_2019,frequentation_2020,frequentation_2021,frequentation_2022,ratio_freq_objets
0,Paris Gare de Lyon,48.844888,2.373520,252766189,123698547,180857682,180857683,1.0
1,Paris Montparnasse,48.841172,2.320514,137996789,73054629,92450093,92450094,1.0
2,Paris Gare du Nord,48.880185,2.355151,517658380,199902302,275471366,275471367,1.0
3,Paris Saint-Lazare,48.876242,2.325331,250460982,267092898,213505115,213505116,10.0
4,Paris Est,48.876742,2.358424,93300618,48823493,62800592,62800593,20.0
5,Paris Austerlitz,48.842285,2.364891,44105915,26806157,45096165,45096166,44.0
6,Paris Bercy,48.839170,2.382780,8800506,5091294,6822335,6822336,132.0
